# Prepare IPyKernel
1. Open Termninal console in Jupyter
2. Create the virtual env: ```python3 -m venv mykernel```
3. Activate env: ```source envs/mykernel/bin/activate```
4. Install ipykernel module: 
```cd envs/mykernel```
```pip install ipykernel```
5. Install / create new kernel
```Documentation: https://ipython.readthedocs.io/en/stable/install/kernel_install.html```
```python -m ipykernel install --user --name mykernel --display-name "Python (mykernel)"```
6. Install Azure Storage Blob & Identity modules: ```pip install azure-storage-blob azure-identity```
7. Switch to Notebook source and select new kernel

# Connect to Azure Blob Storage (applies to managed devices)
1. Open Termninal console in Jupyter
2. Execute: ```az login```
3. Follow the on-screen instructions, sample:
   To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code <code here> to authenticate.
4. Retrieve access token: ```az account get-access-token --output json --resource https://management.core.windows.net```

# Connect to Azure Blob Storage (with client id & certificate)

## Create Private key & SSL Certificate
1. Open Termninal console in Jupyter (or locally if OpenSSL executable is available)
2. Create private key: `openssl genrsa -out myapp_private.key 2048`
    1. Provide private key password
3. Create certificate signing request: `openssl req -new -key myapp_private.key -out myapp.csr`
4. Provide certficate information:
```
       Country Name (2 letter code) [XX]:<your country>
       State or Province Name (full name) []:<your state>
       Locality Name (eg, city) [Default City]:<your city>
       Organization Name (eg, company) [Default Company Ltd]:<your company name>
       Organizational Unit Name (eg, section) []:<your section>
       Common Name (eg, your name or your server's hostname) []:<your name, or team's name>
       Email Address []:
```
5. Generate self-signed certificate: `openssl x509 -req -days 365 -in myapp.csr -signkey myapp_private.key -out myapp_certificate.crt`
6. The .crt file can be handed over to Azure admins, to add it in coresponding App Registration / Certificates & Secrets 

## Export as PKCS12 (includes private key)
1. Open Termninal console in Jupyter (or locally if OpenSSL executable is available)
2. Execute: `openssl pkcs12 -export -out myapp_certificate.pfx -inkey myapp_private.key -in myapp_certificate.crt`
    1. Provide certificate password, it is possible to leave password empty, however it is not recommended
3. The .pfx file (and assigned password) will be used be the client script to access storage account

## Azure Storage Account IAM roles:
1. **Reader** - View all resources, but does not allow you to make any changes (not required)
2. **Storage Blob Data Reader** - Allows for read access to Azure Storage blob containers and data


In [1]:
account_identifier = "storage-account-id"
tenant_id = "<tenant-id>"
client_id = "client-id"
cert_path = "./certificate.pfx"
cert_password = "<passwork>"

container_name = "container001"
folder_path = "Files/"


In [4]:
import os, uuid
from azure.identity import DefaultAzureCredential, CertificateCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

try:
    # If private endpoints are used, make sure no proxy server is used
    #os.environ['no_proxy'] = os.environ['no_proxy'] + f',{account_identifier}.privatelink.blob.core.windows.net,{account_identifier}.blob.core.windows.net'
    
    print("Azure Blob Storage connecting ...")
    account_url = f"https://{account_identifier}.blob.core.windows.net"
    # DefaultAzureCredential will use the credentials provided by 'az login' 
    #default_credential = DefaultAzureCredential()

    cert_credential = CertificateCredential(
       tenant_id=tenant_id,
       client_id=client_id,
       certificate_path=cert_path,
       password=cert_password,
    )

    # Create the BlobServiceClient object
    #blob_service_client = BlobServiceClient(account_url, credential='<storage account key>')
    blob_service_client = BlobServiceClient(account_url, credential=cert_credential)
    print("Azure Blob Storage connected")

    print("\nGetting Blob Container ...")
   
    # Get the container
    container_client = blob_service_client.get_container_client(container_name)
    
    print("\nListing blobs...")
    # List the blobs in the container
    blob_list = container_client.list_blobs(name_starts_with=folder_path)
    for blob in blob_list:
        print("\t" + blob.name)

except Exception as ex:
    print('Exception:')
    print(ex)
    
print("Finished")    

Azure Blob Storage connecting ...
Azure Blob Storage connected

Getting Blob Container ...

Listing blobs...
	Files/dataset1.csv
	Files/dataset2.csv
	Files/dataset3.csv
Finished
